<table>
    <tr>
        <td><img src='SystemLink_icon.png' /></td>
        <td ><h1><strong>NI SystemLink Python API</strong></h1></td>
    </tr>
</table>

## File Service Example
***
The File Service API provides functions to upload, delete, download, update or query files hosted by the File Service.
***
# Prerequisites
- None

# Summary 
This example uses the nptdms package to create a TDMS file with 10 sine waves, with noise. Each waveform contains 1000 points and has a frequency of 20 hertz. The File Service API is used to upload the resulting TDMS file, if the file is not present. The uploaded file is used by the TDMSReader example notebook.

- Use **numpy** to create 10 noisy sine waves.
- Use **nptdms** to create a TDMS file with 1 group (signals) and 10 channels (noisy sine waves)
- Use the File Service API **upload_file** to upload the example file to the File Service.
***

In [1]:
import os
import numpy as np

from nptdms import TdmsWriter, RootObject, GroupObject, ChannelObject
from systemlink.clients.nifile.api.files_api import FilesApi
from systemlink.clients.nifile.models.query_available_files_request import QueryAvailableFilesRequest
from systemlink.clients.nifile.models.property_query import PropertyQuery

root = RootObject(properties={
    'name' : 'sine_wave.tdms',
    'description': 'Test file',
})
group = GroupObject('signals', properties={
    'description': 'Group that holds signals'
})

frequency = 20
sample_rate = 1000

file_name = 'sine_wave_sdk_example.tdms'

if not os.path.exists(file_name):
    files_api = FilesApi()
    property_query = PropertyQuery(key="Name", operation="EQUAL", value=file_name)
    query = QueryAvailableFilesRequest(properties_query=[property_query])
    res = await files_api.query_available_files(query=query)

    if not res.available_files:
        channels = []
        time = np.arange(sample_rate)

        # Create 10 channels, with sine waves + noise (from the standard normal distribution)
        for idx in range(10):
            data = np.sin(2 * np.pi * frequency * time / sample_rate)*5 + np.random.randn(1000)
            channels.append(ChannelObject('signals', 'sine_wave{}'.format(idx), data))

        with TdmsWriter(file_name) as tdms_writer:
            for channel in channels:
                tdms_writer.write_segment([root, group, channel])
        file_id = await files_api.upload(file=file_name)
        print('Uploaded file {} to SystemLink. File ID: {}'.format(file_name, file_id))
        os.remove(file_name)
    else:
        print('File {} is already present in SystemLink. You need to delete the file manually before you can run this example.'.format(file_name))
else:
    print('File {} already exists. You need to delete it manually before this notebook can run.'.format(file_name))

Uploaded file sine_wave_sdk_example.tdms to SystemLink. File ID: {'uri': '/nifile/v1/service-groups/Default/files/61251f3e26b41e2df0eafade'}
